# Differential expression between NPC and NLH

**Top Note:** we should consider normalise the library size (labelled as `exp_sum_count` or `total_counts`) with predefined coefficient weight as 1.

$$y \sim NB(\mu, \phi)$$
$$\log(\mu) = w_0 * 1 + w_1 * condition + 1 * total\_counts$$

You may use your implemement and fit this parameter (it should be close enough to 1) or use 
[statsmodels](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.NegativeBinomial.html)
by setting it as `exposure`:

```Python
NB_glm = sm.discrete.discrete_model.NegativeBinomial(
    obs_y, feature_x,                                                         
    loglike_method = 'nb2', 
    exposure = sample_libsize, 
    offset = None, 
    missing = 'none', 
    check_rank = True
)
```

For negative binomial likelihood, check [aod R package](https://rdrr.io/cran/aod/man/negbin.html).

In [1]:
import numpy as np
import pandas as pd
from scipy.io import mmread

## Load genes

In [2]:
df_genes = pd.read_csv('../data/NPC_NLH-genes.tsv', sep='\t')
df_genes

,gene_names,gene_ids
0,HES5,ENSG00000197921
1,TP73,ENSG00000078900
2,HES2,ENSG00000069812
3,PAX7,ENSG00000009709
4,E2F2,ENSG00000007968
...,...,...
614,OLIG1,ENSG00000184221
615,RUNX1,ENSG00000159216
616,ERG,ENSG00000157554
617,ETS2,ENSG00000157557


## Donor level - T cells

In [3]:
df_donor_T = pd.read_csv('../data/NPC_NLH-Tcell-donorX.tsv', sep='\t', index_col=0)
df_donor_T

,condition,cell_number,total_counts,HES5,TP73,HES2,PAX7,E2F2,RUNX3,TFAP2E,...,TEF,SREBF2,GABPA,BACH1,OLIG2,OLIG1,RUNX1,ERG,ETS2,PKNOX1
patient_ID,,,,,,,,,,,,,,,,,,,,,
1,NPC,1347,4519666.0,0.0,0.0,0.0,0.0,3.0,460.0,6.0,...,6.0,141.0,95.0,124.0,0.0,0.0,177.0,0.0,53.0,76.0
2,NPC,1948,7188569.0,0.0,25.0,0.0,0.0,37.0,1265.0,8.0,...,21.0,159.0,134.0,99.0,0.0,0.0,362.0,0.0,128.0,106.0
3,NPC,3677,5781870.0,0.0,8.0,0.0,0.0,11.0,514.0,5.0,...,10.0,143.0,76.0,88.0,0.0,0.0,283.0,0.0,57.0,64.0
4,NPC,1324,3987246.0,0.0,3.0,0.0,0.0,9.0,603.0,3.0,...,6.0,87.0,80.0,93.0,0.0,0.0,178.0,0.0,45.0,57.0
5,NPC,1567,5188094.0,0.0,15.0,0.0,0.0,37.0,495.0,4.0,...,7.0,193.0,97.0,130.0,0.0,0.0,242.0,0.0,28.0,74.0
9,NPC,4424,20663310.0,1.0,62.0,2.0,0.0,72.0,1971.0,37.0,...,32.0,886.0,410.0,480.0,0.0,0.0,983.0,0.0,226.0,280.0
10,NPC,3320,12679133.0,0.0,13.0,0.0,0.0,25.0,1016.0,11.0,...,32.0,309.0,204.0,178.0,0.0,0.0,545.0,0.0,132.0,156.0
11,NLH,2841,11321356.0,0.0,1.0,0.0,0.0,7.0,657.0,4.0,...,24.0,280.0,191.0,148.0,0.0,0.0,505.0,0.0,121.0,123.0
12,NLH,2393,10087484.0,0.0,3.0,0.0,0.0,16.0,798.0,10.0,...,17.0,319.0,177.0,140.0,0.0,0.0,464.0,0.0,100.0,117.0


In [4]:
X_donor_T = df_donor_T[['condition', 'cell_number', 'total_counts']]
X_donor_T

,condition,cell_number,total_counts
patient_ID,,,
1,NPC,1347,4519666.0
2,NPC,1948,7188569.0
3,NPC,3677,5781870.0
4,NPC,1324,3987246.0
5,NPC,1567,5188094.0
9,NPC,4424,20663310.0
10,NPC,3320,12679133.0
11,NLH,2841,11321356.0
12,NLH,2393,10087484.0


In [5]:
Y_donor_T = df_donor_T[df_donor_T.columns[3:]]
Y_donor_T

,HES5,TP73,HES2,PAX7,E2F2,RUNX3,TFAP2E,MTF1,POU3F1,ZNF684,...,TEF,SREBF2,GABPA,BACH1,OLIG2,OLIG1,RUNX1,ERG,ETS2,PKNOX1
patient_ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,3.0,460.0,6.0,92.0,12.0,28.0,...,6.0,141.0,95.0,124.0,0.0,0.0,177.0,0.0,53.0,76.0
2,0.0,25.0,0.0,0.0,37.0,1265.0,8.0,143.0,43.0,59.0,...,21.0,159.0,134.0,99.0,0.0,0.0,362.0,0.0,128.0,106.0
3,0.0,8.0,0.0,0.0,11.0,514.0,5.0,76.0,32.0,37.0,...,10.0,143.0,76.0,88.0,0.0,0.0,283.0,0.0,57.0,64.0
4,0.0,3.0,0.0,0.0,9.0,603.0,3.0,76.0,20.0,42.0,...,6.0,87.0,80.0,93.0,0.0,0.0,178.0,0.0,45.0,57.0
5,0.0,15.0,0.0,0.0,37.0,495.0,4.0,55.0,20.0,45.0,...,7.0,193.0,97.0,130.0,0.0,0.0,242.0,0.0,28.0,74.0
9,1.0,62.0,2.0,0.0,72.0,1971.0,37.0,386.0,306.0,168.0,...,32.0,886.0,410.0,480.0,0.0,0.0,983.0,0.0,226.0,280.0
10,0.0,13.0,0.0,0.0,25.0,1016.0,11.0,188.0,92.0,119.0,...,32.0,309.0,204.0,178.0,0.0,0.0,545.0,0.0,132.0,156.0
11,0.0,1.0,0.0,0.0,7.0,657.0,4.0,206.0,15.0,61.0,...,24.0,280.0,191.0,148.0,0.0,0.0,505.0,0.0,121.0,123.0
12,0.0,3.0,0.0,0.0,16.0,798.0,10.0,176.0,18.0,69.0,...,17.0,319.0,177.0,140.0,0.0,0.0,464.0,0.0,100.0,117.0


## Single cell level - T cells

In [6]:
Y_cell_T_mat = mmread('../data/NPC_NLH-Tcell-Y.mtx')
Y_cell_T_mat

<24571x619 sparse matrix of type '<class 'numpy.int64'>'
	with 683682 stored elements in COOrdinate format>

In [7]:
Y_cell_T = Y_cell_T_mat.A
Y_cell_T

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
X_cell_T = pd.read_csv('../data/NPC_NLH-Tcell-X.tsv', sep='\t')
X_cell_T

,cell_id,patient_ID,condition,exp_sum_count
0,CACCACTAGGAGTCTG-11,11,NLH,3803.0
1,GACTAACGTCGGCTCA-9,9,NPC,3575.0
2,GCAGCCAGTAGCCTCG-5,5,NPC,547.0
3,CACAAACGTCAGGACA-2,2,NPC,3353.0
4,TACGGTATCTCAACTT-1,1,NPC,2762.0
...,...,...,...,...
24566,CTAATGGGTGACTACT-1,1,NPC,4117.0
24567,CAAGAAAAGGCACATG-3,3,NPC,607.0
24568,CACATTTGTGCTCTTC-10,10,NPC,6192.0
24569,CATGGCGAGAGCAATT-10,10,NPC,4793.0


## Donor level - B cell

In [9]:
df_donor_B = pd.read_csv('../data/NPC_NLH-Bcell-donorX.tsv', sep='\t', index_col=0)
df_donor_B

,condition,cell_number,total_counts,HES5,TP73,HES2,PAX7,E2F2,RUNX3,TFAP2E,...,TEF,SREBF2,GABPA,BACH1,OLIG2,OLIG1,RUNX1,ERG,ETS2,PKNOX1
patient_ID,,,,,,,,,,,,,,,,,,,,,
1,NPC,957,5398577.0,1.0,7.0,0.0,0.0,2.0,213.0,0.0,...,6.0,218.0,105.0,501.0,0.0,0.0,73.0,0.0,13.0,32.0
2,NPC,1577,15724341.0,0.0,18.0,0.0,0.0,24.0,266.0,1.0,...,19.0,178.0,121.0,219.0,0.0,0.0,175.0,0.0,3.0,67.0
3,NPC,1894,5610796.0,0.0,7.0,0.0,0.0,10.0,284.0,4.0,...,4.0,220.0,61.0,188.0,0.0,0.0,58.0,0.0,10.0,38.0
4,NPC,3395,18016842.0,0.0,19.0,0.0,0.0,10.0,723.0,4.0,...,19.0,447.0,248.0,393.0,0.0,0.0,262.0,0.0,30.0,146.0
5,NPC,1652,8721679.0,0.0,25.0,0.0,0.0,40.0,446.0,5.0,...,7.0,358.0,165.0,202.0,0.0,0.0,163.0,1.0,14.0,84.0
9,NPC,3072,24296226.0,0.0,22.0,0.0,0.0,16.0,1030.0,9.0,...,26.0,690.0,376.0,1037.0,0.0,0.0,313.0,0.0,23.0,185.0
10,NPC,2716,12745779.0,0.0,4.0,0.0,0.0,7.0,577.0,5.0,...,23.0,361.0,233.0,317.0,0.0,0.0,280.0,0.0,25.0,100.0
11,NLH,2314,12160524.0,0.0,7.0,0.0,0.0,10.0,523.0,4.0,...,18.0,561.0,190.0,504.0,0.0,0.0,328.0,1.0,30.0,95.0
12,NLH,4149,31006660.0,1.0,29.0,0.0,0.0,22.0,1121.0,5.0,...,33.0,987.0,444.0,1165.0,0.0,0.0,462.0,0.0,29.0,239.0


In [10]:
X_donor_B = df_donor_B[['condition', 'cell_number', 'total_counts']]
X_donor_B

,condition,cell_number,total_counts
patient_ID,,,
1,NPC,957,5398577.0
2,NPC,1577,15724341.0
3,NPC,1894,5610796.0
4,NPC,3395,18016842.0
5,NPC,1652,8721679.0
9,NPC,3072,24296226.0
10,NPC,2716,12745779.0
11,NLH,2314,12160524.0
12,NLH,4149,31006660.0


In [11]:
Y_donor_B = df_donor_B[df_donor_B.columns[3:]]
Y_donor_B

,HES5,TP73,HES2,PAX7,E2F2,RUNX3,TFAP2E,MTF1,POU3F1,ZNF684,...,TEF,SREBF2,GABPA,BACH1,OLIG2,OLIG1,RUNX1,ERG,ETS2,PKNOX1
patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.0,7.0,0.0,0.0,2.0,213.0,0.0,62.0,1.0,19.0,...,6.0,218.0,105.0,501.0,0.0,0.0,73.0,0.0,13.0,32.0
2,0.0,18.0,0.0,0.0,24.0,266.0,1.0,101.0,8.0,38.0,...,19.0,178.0,121.0,219.0,0.0,0.0,175.0,0.0,3.0,67.0
3,0.0,7.0,0.0,0.0,10.0,284.0,4.0,71.0,4.0,12.0,...,4.0,220.0,61.0,188.0,0.0,0.0,58.0,0.0,10.0,38.0
4,0.0,19.0,0.0,0.0,10.0,723.0,4.0,171.0,9.0,71.0,...,19.0,447.0,248.0,393.0,0.0,0.0,262.0,0.0,30.0,146.0
5,0.0,25.0,0.0,0.0,40.0,446.0,5.0,132.0,5.0,45.0,...,7.0,358.0,165.0,202.0,0.0,0.0,163.0,1.0,14.0,84.0
9,0.0,22.0,0.0,0.0,16.0,1030.0,9.0,309.0,20.0,87.0,...,26.0,690.0,376.0,1037.0,0.0,0.0,313.0,0.0,23.0,185.0
10,0.0,4.0,0.0,0.0,7.0,577.0,5.0,183.0,4.0,74.0,...,23.0,361.0,233.0,317.0,0.0,0.0,280.0,0.0,25.0,100.0
11,0.0,7.0,0.0,0.0,10.0,523.0,4.0,227.0,12.0,48.0,...,18.0,561.0,190.0,504.0,0.0,0.0,328.0,1.0,30.0,95.0
12,1.0,29.0,0.0,0.0,22.0,1121.0,5.0,479.0,17.0,102.0,...,33.0,987.0,444.0,1165.0,0.0,0.0,462.0,0.0,29.0,239.0


## Single cell level - B cells

In [12]:
Y_cell_B_mat = mmread('../data/NPC_NLH-Bcell-Y.mtx')
Y_cell_B_mat

<24296x619 sparse matrix of type '<class 'numpy.int64'>'
	with 852687 stored elements in COOrdinate format>

In [13]:
Y_cell_B = Y_cell_B_mat.A
Y_cell_B

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
X_cell_B = pd.read_csv('../data/NPC_NLH-Bcell-X.tsv', sep='\t')
X_cell_B

,cell_id,patient_ID,condition,exp_sum_count
0,CACCACTAGGAGTCTG-11,11,NLH,3803.0
1,GACTAACGTCGGCTCA-9,9,NPC,3575.0
2,GCAGCCAGTAGCCTCG-5,5,NPC,547.0
3,CACAAACGTCAGGACA-2,2,NPC,3353.0
4,TACGGTATCTCAACTT-1,1,NPC,2762.0
...,...,...,...,...
24566,CTAATGGGTGACTACT-1,1,NPC,4117.0
24567,CAAGAAAAGGCACATG-3,3,NPC,607.0
24568,CACATTTGTGCTCTTC-10,10,NPC,6192.0
24569,CATGGCGAGAGCAATT-10,10,NPC,4793.0
